In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import scipy

import tensorflow as tf
from tensorflow.keras import *
from tensorflow.keras.layers import *
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.datasets import fashion_mnist
!pip install ucimlrepo
from ucimlrepo import fetch_ucirepo
from sklearn.model_selection import train_test_split

In [ ]:
# select initial population

# build models
# assemble costs
# convert costs into probability distribution
# compute new population (e.g. using expected value or sampling probability distribution)

# repeat

In [ ]:
# Hypothesis: statistically-significant improvement (needs to be well-defined)

# Control:
# Baseline with random uniform sampling and brute force model building

# Experimental:
# Several cycles of tuning heuristic such that total number of models is less than that of control

In [ ]:
np.random.seed(0)

# Hyperparameter and Baseline Testing Classes, I/O Functions

In [ ]:
class Hyperparameter():
    def __init__(self, sample_func, low, high, dtype):
        self.sample_f = sample_func
        self.low = low
        self.high = high
        self.cast = dtype
    def change_bounds(self, low_new, high_new):
        self.low = low_new
        self.high = high_new
    def sample(self):
        return self.sample_f(self.low, self.high)
    def sample_uniform(self, low, high):
        return np.random.uniform(low=low, high=high, size=None)        

In [ ]:
class Baseline():
    def __init__(self, hparams, n_models, data_func, model_func):
        self.hyperparameters = hparams # dictionary of Hyperparameter objects
        self.num_models = n_models
        self.build_model = model_func
        self.X_t, self.y_t, self.X_v, self.y_v = data_func()
        self.all_hyperparameters = {key: [] for key, value in self.hyperparameters.items()}
        self.losses = []
        self.accuracies = []

    def train_models(self):
        for i in range(self.num_models):
            print("Model", str(i + 1), ": ")
            hparam_values = {}
            for hparam_name, hparam_object in self.hyperparameters.items():
                hparam_value = self.hyperparameters[hparam_name].sample()
                hparam_values[hparam_name] = hparam_value
                self.all_hyperparameters[hparam_name].append(hparam_value)
                print(hparam_name + ": ", str(hparam_value))
            loss, accuracy = self.build_model(hparam_values, self.X_t, self.y_t, self.X_v, self.y_v)
            self.losses.append(loss)
            self.accuracies.append(accuracy)
            print("Loss:", str(loss))
            print("Accuracy:", str(accuracy))
            print()
        result = pd.DataFrame(self.all_hyperparameters).join(pd.DataFrame({"loss": self.losses})).join(pd.DataFrame({"accuracy": self.accuracies}))
        return result

In [ ]:
def prepare_ann_data():
    breast_cancer_wisconsin_diagnostic = fetch_ucirepo(id=17) 
    X = breast_cancer_wisconsin_diagnostic.data.features.values
    y = breast_cancer_wisconsin_diagnostic.data.targets.values
    y[y==["M"]] = 1
    y[y==["B"]] = 0
    y = y.reshape(-1).astype("float32")
    X_t, X_v, y_t, y_v = train_test_split(X, y, test_size=0.2, random_state=0)
    means = np.mean(X_t, axis=0)
    stddevs = np.std(X_t, axis=0)
    X_t = (X_t - means) / stddevs
    X_v = (X_v - means) / stddevs
    return X_t, y_t, X_v, y_v

In [ ]:
def build_ann(hparams_list, X_t, y_t, X_v, y_v):
    learning_rate = 1e-3
    num_units = 16
    dropout_rate = 0.6
    if "learning_rate" in hparams_list:
        learning_rate = hparams_list["learning_rate"]
    if "num_units" in hparams_list:
        num_units = hparams_list["num_units"]
    if "dropout_rate" in hparams_list:
        dropout_rate = hparams_list["dropout_rate"]
    input = Input(shape=(X_t.shape[1:]))
    x = Dense(num_units, activation="relu")(input)
    x = Dropout(dropout_rate)(x)
    x = Dense(4, activation="relu")(x)
    x = Dropout(dropout_rate)(x)
    output = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=input, outputs=output)

    model.compile(optimizer=optimizers.Adam(learning_rate=learning_rate), loss="binary_crossentropy", metrics=["binary_accuracy"])
    history = model.fit(X_t, y_t, validation_data=(X_v, y_v), batch_size=64, epochs=20, verbose=False)
    result = model.evaluate(X_v, y_v, verbose=False)
    loss = result[0]
    accuracy = result[1]
    return loss, accuracy

In [ ]:
def prepare_cnn_data():
    (X_t, y_t), (X_v, y_v) = fashion_mnist.load_data()
    X_t = np.expand_dims(X_t, axis=-1).astype("float32") / 255.0
    X_v = np.expand_dims(X_v, axis=-1).astype("float32") / 255.0
    y_t = to_categorical(y_t)
    y_v = to_categorical(y_v)
    return X_t, y_t, X_v, y_v

In [ ]:
def build_cnn(hparams_list, X_t, y_t, X_v, y_v):
    learning_rate = 1e-3
    num_filters = 32
    if "learning_rate" in hparams_list:
        learning_rate = hparams_list["learning_rate"]
    if "num_filters" in hparams_list:
        num_filters = hparams_list["num_filters"]
    input = Input(shape=(X_t.shape[1:]))
    x = Conv2D(16, (3,3), padding="same", activation="relu")(input)
    x = MaxPooling2D((2,2))(x)
    x = Conv2D(num_filters, (3,3), padding="same", activation="relu")(x)
    x = MaxPooling2D((2,2))(x)
    x = Conv2D(32, (3,3), padding="same", activation="relu")(x)
    x = MaxPooling2D((2,2))(x)
    x = Flatten()(x)
    output = Dense(10, activation="softmax")(x)
    model = Model(inputs=input, outputs=output)

    model.compile(optimizer=optimizers.Adam(learning_rate=learning_rate), loss="categorical_crossentropy", metrics=["categorical_accuracy"])
    history = model.fit(X_t, y_t, validation_data=(X_v, y_v), batch_size=64, epochs=10, verbose=False)
    result = model.evaluate(X_v, y_v, verbose=False)
    loss = result[0]
    accuracy = result[1]
    return loss, accuracy

In [ ]:
def prepare_rnn_data(lookback=None):
    def create_examples(X, y, lookback, pred_size=10, test_size=0.2):
        X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)
        y = (y - np.mean(y)) / np.std(y)
        X_new = []
        Y_new = []
        for i in range(0, X.shape[0] - lookback - pred_size):
            X_new.append(X[i:i + lookback, :])
            Y_new.append(y[i + lookback:i + lookback + pred_size])
        X_new = np.array(X_new)
        Y_new = np.array(Y_new)
        split = X_new.shape[0] - int(test_size * X_new.shape[0])
        X_t = X_new[:split]
        X_v = X_new[split:]
        y_t = Y_new[:split]
        y_v = Y_new[split:]
        return X_t, y_t, X_v, y_v
    if lookback == None:
        return None, None, None, None
    rolling_window = 100
    nth = 5
    air_quality = fetch_ucirepo(id=360) 
    X = air_quality.data.features 
    y = air_quality.data.targets 
    subset = air_quality.data.features[["PT08.S1(CO)","C6H6(GT)","PT08.S2(NMHC)","PT08.S3(NOx)","PT08.S4(NO2)","PT08.S5(O3)","RH","AH","T"]]
    subset = subset[np.sum(subset == -200, axis=1) == 0]
    X = subset.iloc[:,:-1]
    y = subset.iloc[:,-1]
    X = X.rolling(window=rolling_window).mean().iloc[rolling_window:].iloc[::nth].reset_index(drop=True).values
    y = y.rolling(window=rolling_window).mean().iloc[rolling_window:].iloc[::nth].reset_index(drop=True).values
    X_t, y_t, X_v, y_v = create_examples(X, y, lookback=lookback)
    return X_t, y_t, X_v, y_v

In [ ]:
def build_rnn(hparams_list, X_t, y_t, X_v, y_v):
    X_t, y_t, X_v, y_v = prepare_rnn_data(lookback=hparams_list["lookback"])
    num_units = 16
    if "num_units" in hparams_list:
        num_units = hparams_list["num_units"]
    input = Input(shape=(X_t.shape[1:]))
    x = LSTM(num_units, activation="tanh")(input)
    output = Dense(10)(x)
    model = Model(inputs=input, outputs=output)

    model.compile(optimizer=optimizers.Adam(learning_rate=1e-3), loss="mae", metrics=["mse"])
#     es = callbacks.EarlyStopping(monitor="val_loss", mode="min", patience=25, restore_best_weights=True)
    history = model.fit(X_t, y_t, validation_data=(X_v, y_v), batch_size=64, epochs=40, verbose=False)
    result = model.evaluate(X_v, y_v, verbose=False)
    loss = result[0]
    accuracy = result[1]
    return loss, accuracy

# Baseline Testing

In [ ]:
# ANN
num_units = Hyperparameter(lambda l, h: np.random.randint(low=l, high=h), low=1, high=64, dtype=int)
dropout_rate = Hyperparameter(lambda l, h: np.random.uniform(low=l, high=h), low=0.0, high=1.0, dtype=float)

In [ ]:
num_units_dropout_rate_baseline = Baseline(hparams={"num_units": num_units, "dropout_rate": dropout_rate}, 
                                           n_models=50, data_func=prepare_ann_data, model_func=build_ann)
num_units_dropout_rate_results = num_units_dropout_rate_baseline.train_models()

In [ ]:
# CNN
lr = Hyperparameter(lambda l, h: 10 ** np.random.uniform(low=l, high=h), low=-4, high=-1, dtype=float)
num_filters = Hyperparameter(lambda l, h: np.random.randint(low=l, high=h), low=4, high=65, dtype=int)

In [ ]:
lr_num_filters_baseline = Baseline(hparams={"learning_rate": lr, "num_filters": num_filters}, 
                                   n_models=50, data_func=prepare_cnn_data, model_func=build_cnn)
lr_num_filters_results = lr_num_filters_baseline.train_models()

In [ ]:
# RNN
lookback = Hyperparameter(lambda l, h: np.random.randint(low=l, high=h), low=2, high=120, dtype=int)
num_units = Hyperparameter(lambda l, h: np.random.randint(low=l, high=h), low=1, high=128, dtype=int)

In [ ]:
lookback_num_units_baseline = Baseline(hparams={"lookback": lookback, "num_units": num_units}, 
                                       n_models=50, data_func=prepare_rnn_data, model_func=build_rnn)
lookback_num_units_results = lookback_num_units_baseline.train_models()

# Experimental Testing
## Probabilistic Heuristics for Hyperparameter Tuning

In [ ]:
class HyperparameterTuning(Baseline):
    def __init__(self, hparams, n_models, data_func, model_func):
        super().__init__(hparams, n_models, data_func, model_func)
        self.cycle_num = 0
        self.weight = 0.2
        self.num_cycles = 4

    def sample_new(self, losses):
        inverted_losses = 1 - (losses - np.min(losses)) / (np.max(losses) - np.min(losses))
        weighting_factor = 3 # to make probability differences more pronounced
        discrete_pmf = scipy.special.softmax(weighting_factor * inverted_losses)
        return np.random.choice(losses.shape[0], p=discrete_pmf)

    def update_cycle(self):
        self.cycle_num += 1
        self.weight /= 1.5 # weight = 0.2 / (1.5 ** cycle_num)
        self.num_models = int(self.num_models / 1.5)

    def tuning_step(self, result):
        losses = result["loss"].values
        print("Losses:", losses)
        new_hyperparameters = {key: [] for key, value in self.hyperparameters.items()}
        new_losses = []
        new_accuracies = []
        for i in range(self.num_models):
            print("Cycle", str(self.cycle_num), "Model", str(i + 1), ": ")
            if self.num_models == 1:
                idx = np.argmin(losses)
                print("Idx:", idx)
            else:
                idx = self.sample_new(losses)
            hparam_values = {}
            for hparam_name, hparam_object in self.hyperparameters.items():
                hparam_value_mean = result[hparam_name][idx]
                print("Hparam value mean:", hparam_value_mean)
                hparam_value = hparam_object.sample_uniform(hparam_value_mean * (1 - self.weight), hparam_value_mean * (1 + self.weight))
                # hparam_object.change_bounds(hparam_value_mean * (1 - self.weight), hparam_value_mean * (1 + self.weight))
                hparam_value = hparam_object.cast(hparam_value)
                hparam_values[hparam_name] = hparam_value
                new_hyperparameters[hparam_name].append(hparam_value)
                print(hparam_name + ": ", str(hparam_value))
            loss, accuracy = self.build_model(hparam_values, self.X_t, self.y_t, self.X_v, self.y_v)
            new_losses.append(loss)
            new_accuracies.append(accuracy)
            print("Loss: ", str(loss))
            print("Accuracy: ", str(accuracy))
            print()
        new_result = pd.DataFrame({"cycle": [self.cycle_num for i in range(self.num_models)]}).join(pd.DataFrame(new_hyperparameters).join(pd.DataFrame({"loss": new_losses})).join(pd.DataFrame({"accuracy": new_accuracies})))
#         result = result.append(new_result).reset_index(drop=True)
        result = pd.concat([result, new_result], ignore_index=True)
        return result

    def run_heuristic(self):
        result = self.train_models() # train initial population
        result = pd.DataFrame({"cycle": [self.cycle_num for i in range(self.num_models)]}).join(result)
        for i in range(self.num_cycles):
            self.update_cycle()
            result = self.tuning_step(result)
            if self.num_models == 1:
                break
        return result

In [ ]:
# ANN
num_units_dropout_rate_tuning = HyperparameterTuning(hparams={"num_units": num_units, "dropout_rate": dropout_rate}, 
                                                     n_models=10, data_func=prepare_ann_data, model_func=build_ann)
ann_result = num_units_dropout_rate_tuning.run_heuristic()

In [ ]:
# CNN
lr_num_filters_tuning = HyperparameterTuning(hparams={"learning_rate": lr, "num_filters": num_filters}, 
                                             n_models=10, data_func=prepare_cnn_data, model_func=build_cnn)
cnn_result = lr_num_filters_tuning.run_heuristic()

In [ ]:
# RNN
lookback_num_units_tuning = HyperparameterTuning(hparams={"lookback": lookback, "num_units": num_units}, 
                                                 n_models=10, data_func=prepare_rnn_data, model_func=build_rnn)
rnn_result = lookback_num_units_tuning.run_heuristic()

In [ ]:
def plot_results(result):
    plt.plot([np.mean(result[result["cycle"]==i]["loss"]) for i in range(np.max(result["cycle"]) + 1)], linestyle="--", marker="o")
    plt.plot([np.median(result[result["cycle"]==i]["loss"]) for i in range(np.max(result["cycle"]) + 1)], linestyle="--", marker="o")
    plt.plot([np.max(result[result["cycle"]==i]["loss"]) for i in range(np.max(result["cycle"]) + 1)], linestyle="--", marker="o")
    plt.plot([np.min(result[result["cycle"]==i]["loss"]) for i in range(np.max(result["cycle"]) + 1)], linestyle="--", marker="o")
    plt.show()

In [ ]:
def save_csv(file, name):
    file.to_csv("results/" + name + ".csv")